In [35]:
!pip install xgboost
!pip install spacy
!pip install nltk
!pip install GridSearchCV

ERROR: Could not find a version that satisfies the requirement GridSearchCV (from versions: none)
ERROR: No matching distribution found for GridSearchCV


In [52]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
import nltk
from nltk.corpus import stopwords
import re
from collections import Counter
import spacy
from xgboost import XGBClassifier

In [53]:
# Download necessary NLTK data
nltk.download('stopwords')

# Download spaCy French model (run this command in your terminal)
spacy.cli.download("fr_core_news_sm")

# Load spaCy French model
nlp = spacy.load('fr_core_news_sm')

# Load the dataset
file_path = 'projetintegrer.csv'
data = pd.read_csv(file_path)

# Initialize French stopwords
stop_words = set(stopwords.words('french'))

# Function to clean tokenized text
def clean_text(tokens):
    # Remove punctuation and special characters
    tokens = [re.sub(r'\W+', '', token) for token in tokens]
    # Convert to lowercase
    tokens = [token.lower() for token in tokens]
    # Remove stopwords
    tokens = [token for token in tokens if token not in stop_words]
    # Lemmatize using spaCy
    doc = nlp(' '.join(tokens))
    tokens = [token.lemma_ for token in doc]
    # Remove short words
    tokens = [token for token in tokens if len(token) > 2]
    return tokens

# Apply cleaning function to tokenized text
data['Cleaned_Tokens'] = data['Tokenized'].apply(lambda x: clean_text(eval(x)))

# Flatten the list of tokens to create a single list of all words
all_words = [word for tokens in data['Cleaned_Tokens'] for word in tokens]

# Calculate the frequency of each word
word_freq = Counter(all_words)

# Define a threshold for rare words (e.g., words that appear less than 5 times)
threshold = 5

# Create a set of rare words
rare_words = {word for word, freq in word_freq.items() if freq < threshold}

# Function to remove rare words from tokenized text
def remove_rare_words(tokens):
    return [token for token in tokens if token not in rare_words]


# Apply cleaning function to tokenized text
data['Cleaned_Tokens'] = data['Tokenized'].apply(lambda x: clean_text(eval(x)))
data['Cleaned_Text'] = data['Cleaned_Tokens'].apply(lambda x: ' '.join(x))


# Display the cleaned data
print(data[['Tokenized', 'Cleaned_Tokens', 'Cleaned_Text']])

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Administrator\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


✔ Download and installation successful
You can now load the package via spacy.load('fr_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
                                              Tokenized  \
0                             ['lord', 'god', 'angels']   
1     ['blessings', 'iv', 'hated', 'needles', 'young...   
2     ['stage', 'small', 'cell', 'stage', 'years', '...   
3                                             ['delia']   
4                               ['get', 'well', 'soon']   
...                                                 ...   
1886      ['stay', 'strong', 'queen', 'praying', 'fir']   
1887  ['sending', 'prayers', 'may', 'god', 'give', '...   
1888                     ['hey', 'beautiful', 'missed']   
1889                           ['stay', 'strong', 'xx']   
1890          

In [54]:
# Generate dummy sentiment labels for demonstration
import numpy as np
np.random.seed(42)
data['Sentiment'] = np.random.choice([0, 1], size=len(data))

In [55]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data['Cleaned_Text'], data['Sentiment'], test_size=0.2, random_state=42)


In [58]:
# Vectorize the text using TF-IDF
vectorizer = TfidfVectorizer(max_features=5000)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)


In [59]:
# Train and evaluate the best Naive Bayes model
# Train and evaluate Naive Bayes model
nb_model = MultinomialNB()
nb_model.fit(X_train_tfidf, y_train)
nb_pred = nb_model.predict(X_test_tfidf)
nb_accuracy = accuracy_score(y_test, nb_pred)
nb_report = classification_report(y_test, nb_pred)

print("Naive Bayes Accuracy:", nb_accuracy)
print("Naive Bayes Classification Report:\n", nb_report)

Naive Bayes Accuracy: 0.45910290237467016
Naive Bayes Classification Report:
               precision    recall  f1-score   support

           0       0.45      0.68      0.54       176
           1       0.49      0.27      0.35       203

    accuracy                           0.46       379
   macro avg       0.47      0.47      0.44       379
weighted avg       0.47      0.46      0.44       379



In [60]:
# Train and evaluate Random Forest model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train_tfidf, y_train)
rf_pred = rf_model.predict(X_test_tfidf)
rf_accuracy = accuracy_score(y_test, rf_pred)
rf_report = classification_report(y_test, rf_pred)

print("Random Forest Accuracy:", rf_accuracy)
print("Random Forest Classification Report:\n", rf_report)

Random Forest Accuracy: 0.5303430079155673
Random Forest Classification Report:
               precision    recall  f1-score   support

           0       0.49      0.50      0.50       176
           1       0.56      0.56      0.56       203

    accuracy                           0.53       379
   macro avg       0.53      0.53      0.53       379
weighted avg       0.53      0.53      0.53       379



In [61]:
# Train and evaluate XGBoost model
xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb_model.fit(X_train_tfidf, y_train)
xgb_pred = xgb_model.predict(X_test_tfidf)
xgb_accuracy = accuracy_score(y_test, xgb_pred)
xgb_report = classification_report(y_test, xgb_pred)

print("XGBoost Accuracy:", xgb_accuracy)
print("XGBoost Classification Report:\n", xgb_report)

XGBoost Accuracy: 0.5118733509234829
XGBoost Classification Report:
               precision    recall  f1-score   support

           0       0.47      0.45      0.46       176
           1       0.54      0.57      0.55       203

    accuracy                           0.51       379
   macro avg       0.51      0.51      0.51       379
weighted avg       0.51      0.51      0.51       379

